In [ ]:
import wheel
import numpy as np
import sklearn as skl
from mat4py import loadmat
import scipy.io
import csv
import struct
import h5py
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colormaps
#from mne_bids import BIDSPath, read_raw_bids
from scipy.signal import butter, filtfilt,welch
from scipy.stats import zscore 
#from mne.time_frequency import tfr_multitaper
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import tkinter as tk
#import mne
#from mne.viz import plot_alignment, snapshot_brain_montage
#import mne_bids
#from mne.channels import make_standard_montage

from sklearn.model_selection import train_test_split
import pickle
import os
import dask
import dask.dataframe as dd


In [ ]:
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('once')

In [ ]:
#Will be populated with all subjects 
global sub_obj_dict 

In [ ]:
#vis_w2ver = arr_find_in_sentence_v2(trigger_frame, 2, "ver", 2)

def arr_find_in_sentence_v2(trigger_frame, ev_num, w_type, modality) -> list:
    """Aligns triggers and creates np.array 0/1 mapping of which events in trigger_list fit the event pattern type"""
    # Take number of rows in the trigger list
    n_events = len(trigger_frame)
    # Create a binary mapping that starts with every value being false
    sel = np.zeros(n_events, dtype=bool)
    # Looping through all events
    for i in range(n_events):
        #Take the current row 
        event = trigger_frame.iloc[i].to_dict()
        #pull off the sentence dictionary
        sentence = event['sentence']
        #grab the event type
        event_type = event['Type']
        #Go into a switch case like logical statement to determine how to set the mapping
        try:
            if sentence != {} and ('w1Type' in sentence or 'relatedImage' in sentence):
                match = False
                if ev_num == 1 and 'w1Type' in sentence:
                    match = sentence['w1Type'] == w_type and 'WORD1' in event_type
                elif ev_num == 2 and 'w2Type' in sentence:
                    match = sentence['w2Type'] == w_type and 'WORD2' in event_type
                elif ev_num == 3 and 'w3Type' in sentence:
                    match = sentence['w3Type'] == w_type and 'WORD3' in event_type
                elif ev_num == 4 and 'w4Type' in sentence:
                    match = sentence['w4Type'] == w_type and 'WORD4' in event_type
                elif ev_num == 5 and 'sentenceType' in sentence:
                    match = sentence['sentenceType'] == w_type and 'WAIT' in event_type
                elif ev_num == 6 and 'relatedImage' in sentence:
                    match = sentence['relatedImage'] == w_type and 'QUESTION' in event_type
                elif ev_num == 7 and 'Type' in sentence:
                    match = sentence['Type'] == w_type and 'SW_WORD' in event_type
                elif ev_num == 8 and 'SW_HASH' in event_type:
                    match = True

                if match:
                    sel[i] = True
        except KeyError:
            pass
    
    #In the case of an ev_num less than 7 take into count audio or visual within the type field
    if ev_num < 7:
        aud_sel = np.array(['_AUD' in event['Type'] for event in trigger_frame.to_dict('records')])
        vis_sel = np.array(['_VIS' in event['Type'] for event in trigger_frame.to_dict('records')])
        #In modality audio is a and visual is v
        if modality == 1:
            sel = sel & aud_sel
        else:
            sel = sel & vis_sel
   
    return sel.tolist()

In [ ]:
#Global vars for preprocessing
tapers = [3, 4]
params_Hgamma = {
    'fpass': [30, 150],  # Frequency range
    'tapers': tapers,
    'Fs': 1000, # Sampling frequency
    'trialave': 0
}

In [ ]:
def notch_filter(data, fs, freq=60.0, quality_factor=30.0):
    b_notch, a_notch = butter(2, [freq - 1, freq + 1], btype='bandstop', fs=fs)
    return filtfilt(b_notch, a_notch, data)

In [ ]:

def gamma_bandpass_filter(data, fs, low_freq=30, high_freq=150):
    nyquist = 0.5 * fs
    low = low_freq / nyquist
    high = high_freq / nyquist
    b, a = butter(3, [low, high], btype='band')
    return filtfilt(b, a, data)

In [ ]:
def extract_gamma_power(data, fs, low_freq=30, high_freq=150):
    freqs, psd = welch(data, fs, nperseg=fs)
    gamma_mask = (freqs >= low_freq) & (freqs <= high_freq)
    return np.mean(psd[gamma_mask])


In [ ]:
import math

In [ ]:

class Subject:
    """Class to Create subject objects with identifying variables and methods for loading and analysis"""
    def __init__(self, subject, subNum, record_direc):
        self.subInit = subject
        self.subNum = subNum
        self.record_direc = record_direc
        
    def load_behav_mat(self):
        """Used to load in matlab files for behavioral experiment"""
        #Initialize Paths
        self.Gen_Path = "/Volumes/Expansion/4WT/COLLAB-CODE/Alliyah/Sub-Mat-Converted/"+self.subInit+"/"
        Curr_State = self.Gen_Path+self.subInit+"-CurrState.mat"
        Trig_Sentence = self.Gen_Path+self.subInit+"-sentence.mat"
        Trigger = self.Gen_Path+self.subInit+"-trig.mat"
        
        #Loading in neccesary raw files
        self.curr_raw = scipy.io.loadmat(Curr_State,squeeze_me=True, struct_as_record = False, simplify_cells = True)
        self.trig_raw = scipy.io.loadmat(Trigger, squeeze_me=True, struct_as_record = False, simplify_cells = True)
        self.sentence_raw = scipy.io.loadmat(Trig_Sentence, squeeze_me=True, struct_as_record = False, simplify_cells = True)
        
        #Creating seperate data frames of the nested struct fields
        self.presentation_matrix_frame = pd.DataFrame(self.curr_raw['curr_state_export']['presentation_matrix'])
        self.trigger_frame = pd.DataFrame(self.trig_raw['trigg_list'])
        self.file_frame = pd.DataFrame(list(self.curr_raw['curr_state_export']['filenames']))
        
    def convert_behav_mats(self):
        """Used to create processed dataframes from raw matlab files"""
        #First handle curr_state
        curr_state_slice = self.curr_raw['curr_state_export']
        #Create seperate dictionary of all columns that can be resolved to just a constant
        self.constants = list(curr_state_slice.keys())
        #remove the class type fields
        self.constants.remove('trigger_list')
        self.constants.remove('presentation_matrix')
        self.constants.remove('blockList')
        self.constants.remove('filenames')
        #Use dictionary comprehension to only keep primitive type data in final dictionary
        self.constants_frame = {key: curr_state_slice[key] for key in self.constants}

        #Handle trigger_frame
        #replacing MATLAB opaque sentences with properly parsed ones
        sentence_dict = self.sentence_raw['sentence_work']
        self.trigger_frame['sentence'] = sentence_dict

    def load_neuro_mat(self):
        """Load in neurological files"""
        #Load in hdr
        global df_saves
        hdr_path = self.Gen_Path+self.subInit+"-hdr.mat"
        self.hdr_raw = curr_raw = scipy.io.loadmat(hdr_path,squeeze_me=True, struct_as_record = False, simplify_cells = True)

        #Handle record
        record_path = "/Volumes/Expansion/4WT/4WT-analysis/DATA/mainfiles/"+self.record_direc+"/neuralMatfile/FILE1.mat"
        #If nothing catched load
        save_id = self.subInit+"_record"
        
        with h5py.File(record_path, 'r', driver = 'sec2') as f: #memory mapping
            record = f['record'][:] 
        self.record = pd.DataFrame(record)
        del record 
        #Otherwise refer to Cache

    def convert_neuro_mat(self):
        #Handle non-class columns of hdr
        """Convert neurological mat files to dataframes"""
        prim_keys = ['ver', 'patientID', 'recordID', 'startdate', 'starttime', 'bytes', 'records', 'duration', 'ns', 'label', 'labelNew']
            
        hdr_class_dict = {key: value for key, value in self.hdr_raw['hdr'].items() if key not in prim_keys and key not in ['labelNew2', 'labelOld']}

       
       
        self.hdr_frame = pd.DataFrame(hdr_class_dict)
        self.hdr_prim_dict= {key: value for key, value in self.hdr_raw['hdr'].items() if key in prim_keys}
        #placing labels in a seperate list
        self.labels = list(self.hdr_raw['hdr']['label'])

    def pre_process_gamma(self):
        """These can be changed if you want to look into other variables like modality (code a or v) /part of speech etc. 
        See gamma script in 4WT dropbox folder for examples of other valid variable types to examine
        """
        aud_wwS1 = arr_find_in_sentence_v2(self.trigger_frame, 5, "GS", 1)
        aud_wwS2 = arr_find_in_sentence_v2(self.trigger_frame, 5, "GNS", 1)
        aud_wwS3 = arr_find_in_sentence_v2(self.trigger_frame, 5, "NGNS", 1)

        record = self.record.transpose()
        nElecs = len(self.labels)
        fs = self.hdr_frame.frequency[0]
        timeFIXBSL = -400
        timeWord = 1100
        idxFIXBSL = int(np.ceil(timeFIXBSL * fs / 1000))
        idxWord = int(np.ceil(timeWord * fs / 1000))
        sizeTime = idxWord - idxFIXBSL

        # Ensure correct indexing for triggers
        idxFX_S1 = np.where(aud_wwS1)[0] - 5 + 1
        idxFX_S2 = np.where(aud_wwS2)[0] - 5 + 1
        idxFX_S3 = np.where(aud_wwS3)[0] - 5 + 1

        # Initialize arrays for filtered values
        filtered_data_S1 = np.zeros((nElecs, len(idxFX_S1)))
        filtered_data_S2 = np.zeros((nElecs, len(idxFX_S2)))
        filtered_data_S3 = np.zeros((nElecs, len(idxFX_S3)))

        for elecNum in range(nElecs):
            try:
                # Perform bipolar montage
                elecSignal = record.iloc[elecNum, :] - record.iloc[elecNum + 1, :] if elecNum < nElecs - 1 else record.iloc[elecNum, :]
                
                # Apply notch filter and gamma band-pass filter
                elecSignal = gamma_bandpass_filter(notch_filter(elecSignal, fs), fs)

                if np.any(np.isnan(elecSignal)):
                    print(f"NaN values found in electrode {elecNum + 1}")
                    continue

                # Extract gamma power for each condition
                gamma_power_S1 = [extract_gamma_power(elecSignal[self.trigger_frame.iloc[idx]['Var1'] + idxFIXBSL:self.trigger_frame.iloc[idx]['Var1'] + idxWord], fs) 
                                  for idx in idxFX_S1 if self.trigger_frame.iloc[idx]['Var1'] + idxWord < len(elecSignal) and self.trigger_frame.iloc[idx]['Var1'] + idxFIXBSL >= 0]
                gamma_power_S2 = [extract_gamma_power(elecSignal[self.trigger_frame.iloc[idx]['Var1'] + idxFIXBSL:self.trigger_frame.iloc[idx]['Var1'] + idxWord], fs) 
                                  for idx in idxFX_S2 if self.trigger_frame.iloc[idx]['Var1'] + idxWord < len(elecSignal) and self.trigger_frame.iloc[idx]['Var1'] + idxFIXBSL >= 0]
                gamma_power_S3 = [extract_gamma_power(elecSignal[self.trigger_frame.iloc[idx]['Var1'] + idxFIXBSL:self.trigger_frame.iloc[idx]['Var1'] + idxWord], fs) 
                                  for idx in idxFX_S3 if self.trigger_frame.iloc[idx]['Var1'] + idxWord < len(elecSignal) and self.trigger_frame.iloc[idx]['Var1'] + idxFIXBSL >= 0]

                # Store gamma power in the arrays
                filtered_data_S1[elecNum, :len(gamma_power_S1)] = gamma_power_S1
                filtered_data_S2[elecNum, :len(gamma_power_S2)] = gamma_power_S2
                filtered_data_S3[elecNum, :len(gamma_power_S3)] = gamma_power_S3

            except Exception as e:
                print(f"Error processing electrode {elecNum + 1}: {e}")

        # Create DataFrames with filtered data
        filtered_df_S1 = pd.DataFrame(filtered_data_S1, index=self.labels)
        filtered_df_S2 = pd.DataFrame(filtered_data_S2, index=self.labels)
        filtered_df_S3 = pd.DataFrame(filtered_data_S3, index=self.labels)

        return filtered_df_S1, filtered_df_S2, filtered_df_S3
     

# Usage example:
# processor = GammaProcessor(record, trigger_frame, hdr_frame, labels)
# zscored_S1, zscored_S2, zscored_S3 = processor.pre_process_gamma()


    def create_sentence_frame(self):
        """Create frame of just the sentence data"""
        empty_dict = {'w1': '','w2': '','w3': '','w4': '','EN_translation': '','imageFile': '','falseImageFile': '','relatedImage': '','sentenceType': '','w1Type': '','w2Type': '','w3Type': '','w4Type': '','w1SoundFile': '','w2SoundFile': '','w3SoundFile': '','w4SoundFile': '','modality': ''
}
        corr_map = {'W1':'w1','W2':'w2','W3':'w3','W4':'w4','word1type':'w1Type','word2type':'w2Type','word3type':'w3Type','word4type':'w4Type','falseimageFile': 'falseImageFile'}
        #Getting rid of Matlab empty maps within the raw data
        count = 0
        #Un-nesting the dictionaries
        for i in range(len(self.trigger_frame['sentence'])):
            self.trigger_frame.at[i, 'sentence'] = self.trigger_frame.at[i, 'sentence']['sen_field']
        
        #Handle empty maps and rows not fitting the expected sentence dictionary format
        for i in range(len(self.trigger_frame['sentence'])):
            if not isinstance(self.trigger_frame.loc[i, 'sentence'], dict):
                self.trigger_frame.at[i, 'sentence'] = empty_dict
            if 'word' in self.trigger_frame.loc[i, 'sentence'] or 'type' in self.trigger_frame.loc[i, 'sentence']:
                self.trigger_frame.drop(i)
                count+=1

        
        #Create list from column
        sen_list = list(self.trigger_frame['sentence'])
        #print(sen_list[200]) 
        #replace incorrect keys 
        for i,item in enumerate(sen_list):
            for key in corr_map:
                if key in item:
                    sen_list[i][corr_map[key]] = sen_list[i].pop(key)
                sen_list[i]['Type'] = self.trigger_frame['Type']
                    
        sen_frame = pd.DataFrame(sen_list, columns = ['w1', 'w2', 'w3', 'w4', 'imageFile', 'falseImageFile', 'relatedImage', 'sentenceType', 'w1Type', 'w2Type', 'w3Type', 'w4Type', 'modality','Type'])
        sen_frame.dropna(inplace = True, how = 'any')
        return sen_frame    

In [ ]:
directory = ["s1_ND_EN","s2_TW","s3_BCH_EN","s4_BWH_EN","s5_BWH_EN","s6_TW_leftHanded_male","s7_TW_rightHandedMale","s8_ccf_SANN001","s9_ccf_SANN002","s10_ccf_SANN003","s11_ccf_SANN004","s12_ccf_SANN005", "s13_ccf_SANN006","s14_ccf_SANN007","s15_ccf_SANN009","s16_ccf_SANN010","s17_ccf_SANN011"]

In [ ]:
def create_sub_dict() -> dict:
    """Loads a dictionary of subject objects to simplify variable access subject_obj_dict[S#] <- access pattern"""
    sub_obj_dict = {}
    for i in range(1,18):
        #Initialize new subject
        curr_sub = f"S{i}"
        curr_direc = directory[i-1]
        sub_obj_dict[curr_sub] = Subject(curr_sub,i,curr_direc)
        sub_obj_dict[curr_sub].load_behav_mat()
        sub_obj_dict[curr_sub].convert_behav_mats()
    return sub_obj_dict

In [ ]:
#dictionary used to store subject objects
#consider caching this
sub_obj_dict = create_sub_dict()

In [ ]:
def sentence_pool() -> pd.DataFrame:
    """Creates a pooled and cleaned dataframe of all the behavioral sentence data with identifying subject number per column"""
    dataframes = []
    for i,sub in enumerate(sub_obj_dict):
        curr_sentence = sub_obj_dict[sub].create_sentence_frame()
        curr_sentence['sub_id'] = i+1
        print(sub)
        print(curr_sentence.shape)
        dataframes.append(curr_sentence)
        print("Dataframe Created and Appended")
        print() 
    total_sentence = pd.concat(dataframes, ignore_index = True)
    return total_sentence

In [ ]:

sentence_input = sentence_pool() 

In [ ]:
#split into test and train sets

In [ ]:
sentence_input

In [ ]:
sentence_input.to_csv('sentence_data.csv.gz', compression = 'gzip') #Save to file

In [ ]:
#Pool neural data
def neural_pool(sub_lst):
    """Loads in, pre-processes, and pools all the SEEG data into a flat dataframe with identifying subject number"""
    gs_lst = []
    gns_lst = []
    ngns_lst = [] 
    for sub in sub_lst:
        #Load the neural data
        print("On Subject ", sub)
        print("Loading and Converting neural data")
        sub_obj_dict[sub].load_neuro_mat()
        sub_obj_dict[sub].convert_neuro_mat()
        #Pre-Process the neural data
        print("Processing Gamma")
        GS_record, GNS_record, NGNS_record= sub_obj_dict[sub].pre_process_gamma()
        print("List appending")
        gs_lst.append(GS_record)
        gns_lst.append(GNS_record)
        ngns_lst.append(NGNS_record)
    #Combine all the frames together
    print("Concating")
    total_gs = pd.concat(gs_lst, ignore_index = True)
    total_gns = pd.concat(gns_lst, ignore_index = True)
    total_ngns = pd.concat(ngns_lst, ignore_index = True)
    return total_gs,total_gns,total_ngns

In [ ]:
iter_lst = list(sub_obj_dict.keys())
iter_lst.pop(2)
iter_lst

In [ ]:
#Change start arguement to do one at a time
for i,sub in enumerate(iter_lst, start = 12):
    gs,gns,ngns = neural_pool([iter_lst[i]])
    file_1= "GS_"+sub+".csv.gz"
    file_2 = "GNS_"+sub+".csv.gz"
    file_3 = "NGNS_"+sub+".csv.gz"
    print("Generating CSV zipped files")
    gs.to_csv(file_1, compression = 'gzip')
    gns.to_csv(file_2, compression = 'gzip')
    ngns.to_csv(file_3, compression = 'gzip')
    print("Garbage Collecting")
    del gs
    del gns
    del ngns
    print("Done!") 
    break 

In [ ]:
with open(filename, 'rb') as csvfile:
    df = pd.read_csv(csvfile,compression='gzip')